In [6]:
import pandas as pd
import numpy as np

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GroupKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score

import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
import polars as pl
from pathlib import Path
from glob import glob
import json
import joblib
import warnings
warnings.filterwarnings("ignore")




In [7]:
with open('../IND-WhoIsWho/pid_to_info_all.json', 'r') as file:
    pid = json.load(file)

In [8]:
data = pd.read_feather('all_feat.feather')

In [9]:


df_train = data[data['label'].notna()].reset_index(drop = True)
df_test = data[data['label'].isna()].reset_index(drop = True)
df_train['label'] = 1 - df_train['label']

drop_feat =['tfidf_autherName','dataset_mode','id'] + ['PID','label','autherID','pid_title','autherName', 'abstract', 'keywords', 'venue', 'authors', 'index','cos_sim_pid','autherName_count'] 
used_feat = [f for f in df_train.columns if f not in (['PID','label','autherID'] + drop_feat )]
print(len(used_feat))
print(used_feat)
ycol = 'label'

#del data
gc.collect()

372
['bge_m3_pid_title_sim_mean', 'bge_m3_pid_title_sim_max', 'bge_m3_pid_title_sim_min', 'bge_m3_pid_title_sim_std', 'bge_m3_pid_title_sim_median', 'bge_m3_pid_abstract_sim_mean', 'bge_m3_pid_abstract_sim_max', 'bge_m3_pid_abstract_sim_min', 'bge_m3_pid_abstract_sim_std', 'bge_m3_pid_abstract_sim_median', 'tfidf_pid_abstract_sim_mean', 'tfidf_pid_abstract_sim_max', 'tfidf_pid_abstract_sim_min', 'tfidf_pid_abstract_sim_std', 'tfidf_pid_abstract_sim_median', 'tfidf_pid_title_sim_mean', 'tfidf_pid_title_sim_max', 'tfidf_pid_title_sim_min', 'tfidf_pid_title_sim_std', 'tfidf_pid_title_sim_median', 'tfidf_pid_venue_sim_mean', 'tfidf_pid_venue_sim_max', 'tfidf_pid_venue_sim_min', 'tfidf_pid_venue_sim_std', 'tfidf_pid_venue_sim_median', 'tfidf_pid_keyword_text_sim_mean', 'tfidf_pid_keyword_text_sim_max', 'tfidf_pid_keyword_text_sim_min', 'tfidf_pid_keyword_text_sim_std', 'tfidf_pid_keyword_text_sim_median', 'jaccard_pid_keyword_cnt_1_mean', 'jaccard_pid_keyword_cnt_1_max', 'jaccard_pid_keywor

0

In [10]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedGroupKFold  # type: ignore


df_importance_dict = dict()
df_oof = df_train[['PID','autherID', 'label']].copy()
prediction = df_test[['PID','autherID']]




#iter_dict = dict({'CHANNEL_A':4000,'CHANNEL_B':10000,'CHANNEL_C':2500})
for ycol in ['label']:
#for ycol in [ 'CHANNEL_C']:

    print(ycol)
    #try:
    #    used_feat = feature_dict[ycol].copy()
    #except:
    #    used_feat = list(feature_dict[ycol])[0]
    print(len(used_feat))
    
    
    df_oof[f"{ycol}_prob"] = 0
    prediction[f"{ycol}_prob"] = 0

    
    df_importance_list = []

    folds = 10
    seeds =[2020]#,2019,2003,2011,2017, 2027]
    for seed in seeds:

        model2 =  LGBMClassifier(objective = 'binary',boosting_type='gbdt',num_leaves=34,max_depth=15,learning_rate=0.01,n_estimators=3500,subsample=0.8,
                               feature_fraction=0.8,reg_alpha=10,reg_lambda= 12,random_state=seed,is_unbalance=True,metric='logloss',n_jobs=24,bagging_freq=1)
        
        
        
        
        #model2 = XGBClassifier(boosting_type='gbdt',num_leaves=15,max_depth=6,learning_rate=0.01,n_estimators=3500,subsample=0.8,tree_method = 'gpu_hist',
        #                   feature_fraction=0.7,random_state=seed,is_unbalance=True,eval_metric='auc')
        """
        ctb_params = dict(iterations=3000,
                          learning_rate=0.01,
                          depth=8,
                          #cat_features = [],
                          l2_leaf_reg=30,
                          bootstrap_type='Bernoulli',
                          subsample=0.8,
                          loss_function='Logloss',
                          #eval_metric = 'AUC',
                          metric_period=100,
                          od_type='Iter',
                          od_wait=30,
                          task_type='GPU',
                          allow_writing_files=False,
                          )

        print("Feature Elimination Performing.")
        model2 = CatBoostClassifier(**ctb_params)
        """
        print("================{}==============".format(seed))
        #kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        kfold = StratifiedGroupKFold(n_splits=folds)
        
        for fold_id, (trn_idx, val_idx) in enumerate(
                kfold.split(df_train[used_feat], df_train[ycol],groups=df_train['autherID'])):
        #for fold_id, (trn_idx, val_idx) in enumerate(
        #        kfold.split(df_train[used_feat], df_train[ycol],groups = df_train['autherID'])):
            X_train = df_train.iloc[trn_idx][used_feat]
            Y_train = df_train.iloc[trn_idx][ycol]

            X_val = df_train.iloc[val_idx][used_feat]
            Y_val = df_train.iloc[val_idx][ycol]
            #df_train.iloc[val_idx,'fold_id'] = fold_id

            print('\nFold_{} Training ================================\n'.format(fold_id + 1))


            lgb_model = model2.fit(X_train[used_feat],Y_train,
                              eval_set=[(X_val[used_feat], Y_val)],
                              #eval_metric = 'auc',
                              verbose=100,
                              #early_stopping_rounds=200
                                  )



            pred_val = lgb_model.predict_proba(X_val[used_feat])[:,1]


            df_oof.loc[val_idx, f"{ycol}_prob"  ] += pred_val


            pred_test = lgb_model.predict_proba(df_test[used_feat])[:,1]
            prediction[f"{ycol}_prob" ] += pred_test

            df_importance = pd.DataFrame({
                'column': used_feat,
                'importance': lgb_model.feature_importances_,
            })
            df_importance_list.append(df_importance)
            #break

            del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
            gc.collect()
            #break
            
       
        df_importance_dict[ycol] = df_importance_list.copy()
    #break   
LABEL_COLUME = ['label']  
prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]]/(folds*len(seeds))

df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] / len(seeds)

label
372
================2020==============

Fold_1 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1

Fold_2 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1

Fold_3 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1

Fold_4 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.8, colsa

ValueError: Input data must be 2 dimensional and non empty.

In [ ]:
df_oof.to_feather('oof2/lgb_oof.feather')
prediction.to_feather('oof2/lgb_prediction.feather')

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedGroupKFold  # type: ignore


df_importance_dict = dict()
df_oof = df_train[['PID','autherID', 'label']].copy()
prediction = df_test[['PID','autherID']]




#iter_dict = dict({'CHANNEL_A':4000,'CHANNEL_B':10000,'CHANNEL_C':2500})
for ycol in ['label']:
#for ycol in [ 'CHANNEL_C']:

    print(ycol)
    #try:
    #    used_feat = feature_dict[ycol].copy()
    #except:
    #    used_feat = list(feature_dict[ycol])[0]
    print(len(used_feat))
    
    
    df_oof[f"{ycol}_prob"] = 0
    prediction[f"{ycol}_prob"] = 0

    
    df_importance_list = []

    folds = 10
    seeds =[2020]#,2019,2003,2011,2017, 2027]
    for seed in seeds:

        #model2 =  LGBMClassifier(objective = 'binary',boosting_type='gbdt',num_leaves=34,max_depth=15,learning_rate=0.01,n_estimators=3500,subsample=0.8,
        #                       feature_fraction=0.8,reg_alpha=10,reg_lambda= 12,random_state=seed,is_unbalance=True,metric='logloss',n_jobs=24,bagging_freq=1)
        
        
        
        
        model2 = XGBClassifier(boosting_type='gbdt',num_leaves=15,max_depth=6,learning_rate=0.01,n_estimators=3500,subsample=0.8,tree_method = 'gpu_hist',
                           feature_fraction=0.7,random_state=seed,is_unbalance=True,eval_metric='auc')
        """
        ctb_params = dict(iterations=3000,
                          learning_rate=0.01,
                          depth=8,
                          #cat_features = [],
                          l2_leaf_reg=30,
                          bootstrap_type='Bernoulli',
                          subsample=0.8,
                          loss_function='Logloss',
                          #eval_metric = 'AUC',
                          metric_period=100,
                          od_type='Iter',
                          od_wait=30,
                          task_type='GPU',
                          allow_writing_files=False,
                          )

        print("Feature Elimination Performing.")
        model2 = CatBoostClassifier(**ctb_params)
        """
        print("================{}==============".format(seed))
        #kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        kfold = StratifiedGroupKFold(n_splits=folds)
        
        for fold_id, (trn_idx, val_idx) in enumerate(
                kfold.split(df_train[used_feat], df_train[ycol],groups=df_train['autherID'])):
        #for fold_id, (trn_idx, val_idx) in enumerate(
        #        kfold.split(df_train[used_feat], df_train[ycol],groups = df_train['autherID'])):
            X_train = df_train.iloc[trn_idx][used_feat]
            Y_train = df_train.iloc[trn_idx][ycol]

            X_val = df_train.iloc[val_idx][used_feat]
            Y_val = df_train.iloc[val_idx][ycol]
            #df_train.iloc[val_idx,'fold_id'] = fold_id

            print('\nFold_{} Training ================================\n'.format(fold_id + 1))


            lgb_model = model2.fit(X_train[used_feat],Y_train,
                              eval_set=[(X_val[used_feat], Y_val)],
                              #eval_metric = 'auc',
                              verbose=100,
                              #early_stopping_rounds=200
                                  )



            pred_val = lgb_model.predict_proba(X_val[used_feat])[:,1]


            df_oof.loc[val_idx, f"{ycol}_prob"  ] += pred_val


            pred_test = lgb_model.predict_proba(df_test[used_feat])[:,1]
            prediction[f"{ycol}_prob" ] += pred_test

            df_importance = pd.DataFrame({
                'column': used_feat,
                'importance': lgb_model.feature_importances_,
            })
            df_importance_list.append(df_importance)
            #break

            del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
            gc.collect()
            #break
            
       
        df_importance_dict[ycol] = df_importance_list.copy()
    #break   
LABEL_COLUME = ['label']  
prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]]/(folds*len(seeds))

df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] / len(seeds)

In [ ]:
df_oof.to_feather('oof2/xgb_oof.feather')
prediction.to_feather('oof2/xgb_prediction.feather')

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedGroupKFold  # type: ignore


df_importance_dict = dict()
df_oof = df_train[['PID','autherID', 'label']].copy()
prediction = df_test[['PID','autherID']]




#iter_dict = dict({'CHANNEL_A':4000,'CHANNEL_B':10000,'CHANNEL_C':2500})
for ycol in ['label']:
#for ycol in [ 'CHANNEL_C']:

    print(ycol)
    #try:
    #    used_feat = feature_dict[ycol].copy()
    #except:
    #    used_feat = list(feature_dict[ycol])[0]
    print(len(used_feat))
    
    
    df_oof[f"{ycol}_prob"] = 0
    prediction[f"{ycol}_prob"] = 0

    
    df_importance_list = []

    folds = 10
    seeds =[2020]#,2019,2003,2011,2017, 2027]
    for seed in seeds:

        #model2 =  LGBMClassifier(objective = 'binary',boosting_type='gbdt',num_leaves=34,max_depth=15,learning_rate=0.01,n_estimators=3500,subsample=0.8,
        #                       feature_fraction=0.8,reg_alpha=10,reg_lambda= 12,random_state=seed,is_unbalance=True,metric='logloss',n_jobs=24,bagging_freq=1)
        
        
        
        
        #model2 = XGBClassifier(boosting_type='gbdt',num_leaves=15,max_depth=6,learning_rate=0.01,n_estimators=3500,subsample=0.8,tree_method = 'gpu_hist',
        #                   feature_fraction=0.7,random_state=seed,is_unbalance=True,eval_metric='auc')
        
        ctb_params = dict(iterations=3000,
                          learning_rate=0.01,
                          depth=8,
                          #cat_features = [],
                          l2_leaf_reg=30,
                          bootstrap_type='Bernoulli',
                          subsample=0.8,
                          loss_function='Logloss',
                          #eval_metric = 'AUC',
                          metric_period=100,
                          od_type='Iter',
                          od_wait=30,
                          task_type='GPU',
                          allow_writing_files=False,
                          )

        print("Feature Elimination Performing.")
        model2 = CatBoostClassifier(**ctb_params)
        
        print("================{}==============".format(seed))
        #kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        kfold = StratifiedGroupKFold(n_splits=folds)
        
        for fold_id, (trn_idx, val_idx) in enumerate(
                kfold.split(df_train[used_feat], df_train[ycol],groups=df_train['autherID'])):
        #for fold_id, (trn_idx, val_idx) in enumerate(
        #        kfold.split(df_train[used_feat], df_train[ycol],groups = df_train['autherID'])):
            X_train = df_train.iloc[trn_idx][used_feat]
            Y_train = df_train.iloc[trn_idx][ycol]

            X_val = df_train.iloc[val_idx][used_feat]
            Y_val = df_train.iloc[val_idx][ycol]
            #df_train.iloc[val_idx,'fold_id'] = fold_id

            print('\nFold_{} Training ================================\n'.format(fold_id + 1))


            lgb_model = model2.fit(X_train[used_feat],Y_train,
                              eval_set=[(X_val[used_feat], Y_val)],
                              #eval_metric = 'auc',
                              verbose=100,
                              #early_stopping_rounds=200
                                  )



            pred_val = lgb_model.predict_proba(X_val[used_feat])[:,1]


            df_oof.loc[val_idx, f"{ycol}_prob"  ] += pred_val


            pred_test = lgb_model.predict_proba(df_test[used_feat])[:,1]
            prediction[f"{ycol}_prob" ] += pred_test

            df_importance = pd.DataFrame({
                'column': used_feat,
                'importance': lgb_model.feature_importances_,
            })
            df_importance_list.append(df_importance)
            #break

            del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
            gc.collect()
            #break
            
       
        df_importance_dict[ycol] = df_importance_list.copy()
    #break   
LABEL_COLUME = ['label']  
prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]]/(folds*len(seeds))

df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] / len(seeds)

In [ ]:
df_oof.to_feather('oof2/cat_oof.feather')
prediction.to_feather('oof2/cat_prediction.feather')

In [28]:
data = pd.read_feather('all_feats2.feather')


df_train = data[data['label'].notna()].reset_index(drop = True)
df_test = data[data['label'].isna()].reset_index(drop = True)
df_train['label'] = 1 - df_train['label']

drop_feat =['tfidf_autherName','dataset_mode','id'] + ['PID','label','autherID','pid_title','autherName', 'abstract', 'keywords', 'venue', 'authors', 'index','cos_sim_pid','autherName_count'] 
used_feat = [f for f in df_train.columns if f not in (['PID','label','autherID'] + drop_feat )]
print(len(used_feat))
print(used_feat)
ycol = 'label'

#del data
gc.collect()

475
['bge_m3_pid_title_sim_mean', 'bge_m3_pid_title_sim_max', 'bge_m3_pid_title_sim_min', 'bge_m3_pid_title_sim_std', 'bge_m3_pid_title_sim_median', 'bge_m3_pid_abstract_sim_mean', 'bge_m3_pid_abstract_sim_max', 'bge_m3_pid_abstract_sim_min', 'bge_m3_pid_abstract_sim_std', 'bge_m3_pid_abstract_sim_median', 'tfidf_pid_abstract_sim_mean', 'tfidf_pid_abstract_sim_max', 'tfidf_pid_abstract_sim_min', 'tfidf_pid_abstract_sim_std', 'tfidf_pid_abstract_sim_median', 'tfidf_pid_title_sim_mean', 'tfidf_pid_title_sim_max', 'tfidf_pid_title_sim_min', 'tfidf_pid_title_sim_std', 'tfidf_pid_title_sim_median', 'tfidf_pid_venue_sim_mean', 'tfidf_pid_venue_sim_max', 'tfidf_pid_venue_sim_min', 'tfidf_pid_venue_sim_std', 'tfidf_pid_venue_sim_median', 'tfidf_pid_keyword_text_sim_mean', 'tfidf_pid_keyword_text_sim_max', 'tfidf_pid_keyword_text_sim_min', 'tfidf_pid_keyword_text_sim_std', 'tfidf_pid_keyword_text_sim_median', 'jaccard_pid_keyword_cnt_1_mean', 'jaccard_pid_keyword_cnt_1_max', 'jaccard_pid_keywor

1207

In [29]:
used_feat = joblib.load('sub/feat_select_lgb_feats.pkl')

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedGroupKFold  # type: ignore


df_importance_dict = dict()
df_oof = df_train[['PID','autherID', 'label']].copy()
prediction = df_test[['PID','autherID']]




#iter_dict = dict({'CHANNEL_A':4000,'CHANNEL_B':10000,'CHANNEL_C':2500})
for ycol in ['label']:
#for ycol in [ 'CHANNEL_C']:

    print(ycol)
    #try:
    #    used_feat = feature_dict[ycol].copy()
    #except:
    #    used_feat = list(feature_dict[ycol])[0]
    print(len(used_feat))
    
    
    df_oof[f"{ycol}_prob"] = 0
    prediction[f"{ycol}_prob"] = 0

    
    df_importance_list = []

    folds = 10
    seeds =[2020]#,2019,2003,2011,2017, 2027]
    for seed in seeds:

        model2 =  LGBMClassifier(objective = 'binary',boosting_type='gbdt',num_leaves=34,max_depth=15,learning_rate=0.01,n_estimators=3500,subsample=0.8,
                               feature_fraction=0.8,reg_alpha=10,reg_lambda= 12,random_state=seed,is_unbalance=True,metric='logloss',n_jobs=24,bagging_freq=1)
        
        
        
        
        #model2 = XGBClassifier(boosting_type='gbdt',num_leaves=15,max_depth=6,learning_rate=0.01,n_estimators=3500,subsample=0.8,tree_method = 'gpu_hist',
        #                   feature_fraction=0.7,random_state=seed,is_unbalance=True,eval_metric='auc')
        """
        ctb_params = dict(iterations=3000,
                          learning_rate=0.01,
                          depth=8,
                          #cat_features = [],
                          l2_leaf_reg=30,
                          bootstrap_type='Bernoulli',
                          subsample=0.8,
                          loss_function='Logloss',
                          #eval_metric = 'AUC',
                          metric_period=100,
                          od_type='Iter',
                          od_wait=30,
                          task_type='GPU',
                          allow_writing_files=False,
                          )

        print("Feature Elimination Performing.")
        model2 = CatBoostClassifier(**ctb_params)
        """
        print("================{}==============".format(seed))
        #kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        kfold = StratifiedGroupKFold(n_splits=folds)
        
        for fold_id, (trn_idx, val_idx) in enumerate(
                kfold.split(df_train[used_feat], df_train[ycol],groups=df_train['autherID'])):
        #for fold_id, (trn_idx, val_idx) in enumerate(
        #        kfold.split(df_train[used_feat], df_train[ycol],groups = df_train['autherID'])):
            X_train = df_train.iloc[trn_idx][used_feat]
            Y_train = df_train.iloc[trn_idx][ycol]

            X_val = df_train.iloc[val_idx][used_feat]
            Y_val = df_train.iloc[val_idx][ycol]
            #df_train.iloc[val_idx,'fold_id'] = fold_id

            print('\nFold_{} Training ================================\n'.format(fold_id + 1))


            lgb_model = model2.fit(X_train[used_feat],Y_train,
                              eval_set=[(X_val[used_feat], Y_val)],
                              #eval_metric = 'auc',
                              verbose=100,
                              #early_stopping_rounds=200
                                  )



            pred_val = lgb_model.predict_proba(X_val[used_feat])[:,1]


            df_oof.loc[val_idx, f"{ycol}_prob"  ] += pred_val


            pred_test = lgb_model.predict_proba(df_test[used_feat])[:,1]
            prediction[f"{ycol}_prob" ] += pred_test

            df_importance = pd.DataFrame({
                'column': used_feat,
                'importance': lgb_model.feature_importances_,
            })
            df_importance_list.append(df_importance)
            #break

            del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
            gc.collect()
            #break
            
       
        df_importance_dict[ycol] = df_importance_list.copy()
    #break   
LABEL_COLUME = ['label']  
prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]]/(folds*len(seeds))

df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] / len(seeds)

label
136
================2020==============

Fold_1 Training ================================


Fold_2 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1

Fold_3 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1

Fold_4 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1

Fold_5 Training ================================

[Lig

ValueError: Input data must be 2 dimensional and non empty.

In [30]:
df_oof.to_feather('oof2/feat_select_lgb_oof.feather')
prediction.to_feather('oof2/feat_select_lgb_prediction.feather')

OSError: Cannot save file into a non-existent directory: 'oof2'

In [ ]:
used_feat = joblib.load('sub/feat_select_xgb_feats.pkl')

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedGroupKFold  # type: ignore


df_importance_dict = dict()
df_oof = df_train[['PID','autherID', 'label']].copy()
prediction = df_test[['PID','autherID']]




#iter_dict = dict({'CHANNEL_A':4000,'CHANNEL_B':10000,'CHANNEL_C':2500})
for ycol in ['label']:
#for ycol in [ 'CHANNEL_C']:

    print(ycol)
    #try:
    #    used_feat = feature_dict[ycol].copy()
    #except:
    #    used_feat = list(feature_dict[ycol])[0]
    print(len(used_feat))
    
    
    df_oof[f"{ycol}_prob"] = 0
    prediction[f"{ycol}_prob"] = 0

    
    df_importance_list = []

    folds = 10
    seeds =[2020]#,2019,2003,2011,2017, 2027]
    for seed in seeds:

        #model2 =  LGBMClassifier(objective = 'binary',boosting_type='gbdt',num_leaves=34,max_depth=15,learning_rate=0.01,n_estimators=3500,subsample=0.8,
        #                       feature_fraction=0.8,reg_alpha=10,reg_lambda= 12,random_state=seed,is_unbalance=True,metric='logloss',n_jobs=24,bagging_freq=1)
        
        
        
        
        model2 = XGBClassifier(boosting_type='gbdt',num_leaves=15,max_depth=6,learning_rate=0.01,n_estimators=3500,subsample=0.8,tree_method = 'gpu_hist',
                           feature_fraction=0.7,random_state=seed,is_unbalance=True,eval_metric='auc')
        """
        ctb_params = dict(iterations=3000,
                          learning_rate=0.01,
                          depth=8,
                          #cat_features = [],
                          l2_leaf_reg=30,
                          bootstrap_type='Bernoulli',
                          subsample=0.8,
                          loss_function='Logloss',
                          #eval_metric = 'AUC',
                          metric_period=100,
                          od_type='Iter',
                          od_wait=30,
                          task_type='GPU',
                          allow_writing_files=False,
                          )

        print("Feature Elimination Performing.")
        model2 = CatBoostClassifier(**ctb_params)
        """
        print("================{}==============".format(seed))
        #kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        kfold = StratifiedGroupKFold(n_splits=folds)
        
        for fold_id, (trn_idx, val_idx) in enumerate(
                kfold.split(df_train[used_feat], df_train[ycol],groups=df_train['autherID'])):
        #for fold_id, (trn_idx, val_idx) in enumerate(
        #        kfold.split(df_train[used_feat], df_train[ycol],groups = df_train['autherID'])):
            X_train = df_train.iloc[trn_idx][used_feat]
            Y_train = df_train.iloc[trn_idx][ycol]

            X_val = df_train.iloc[val_idx][used_feat]
            Y_val = df_train.iloc[val_idx][ycol]
            #df_train.iloc[val_idx,'fold_id'] = fold_id

            print('\nFold_{} Training ================================\n'.format(fold_id + 1))


            lgb_model = model2.fit(X_train[used_feat],Y_train,
                              eval_set=[(X_val[used_feat], Y_val)],
                              #eval_metric = 'auc',
                              verbose=100,
                              #early_stopping_rounds=200
                                  )



            pred_val = lgb_model.predict_proba(X_val[used_feat])[:,1]


            df_oof.loc[val_idx, f"{ycol}_prob"  ] += pred_val


            pred_test = lgb_model.predict_proba(df_test[used_feat])[:,1]
            prediction[f"{ycol}_prob" ] += pred_test

            df_importance = pd.DataFrame({
                'column': used_feat,
                'importance': lgb_model.feature_importances_,
            })
            df_importance_list.append(df_importance)
            #break

            del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
            gc.collect()
            #break
            
       
        df_importance_dict[ycol] = df_importance_list.copy()
    #break   
LABEL_COLUME = ['label']  
prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]]/(folds*len(seeds))

df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] / len(seeds)

In [ ]:
df_oof.to_feather('oof2/feat_select_xgb_oof.feather')
prediction.to_feather('oof2/feat_select_xgb_prediction.feather')

In [ ]:
used_feat = joblib.load('sub/feat_select_cat_feats.pkl')

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedGroupKFold  # type: ignore


df_importance_dict = dict()
df_oof = df_train[['PID','autherID', 'label']].copy()
prediction = df_test[['PID','autherID']]




#iter_dict = dict({'CHANNEL_A':4000,'CHANNEL_B':10000,'CHANNEL_C':2500})
for ycol in ['label']:
#for ycol in [ 'CHANNEL_C']:

    print(ycol)
    #try:
    #    used_feat = feature_dict[ycol].copy()
    #except:
    #    used_feat = list(feature_dict[ycol])[0]
    print(len(used_feat))
    
    
    df_oof[f"{ycol}_prob"] = 0
    prediction[f"{ycol}_prob"] = 0

    
    df_importance_list = []

    folds = 10
    seeds =[2020]#,2019,2003,2011,2017, 2027]
    for seed in seeds:

        #model2 =  LGBMClassifier(objective = 'binary',boosting_type='gbdt',num_leaves=34,max_depth=15,learning_rate=0.01,n_estimators=3500,subsample=0.8,
        #                       feature_fraction=0.8,reg_alpha=10,reg_lambda= 12,random_state=seed,is_unbalance=True,metric='logloss',n_jobs=24,bagging_freq=1)
        
        
        
        
        #model2 = XGBClassifier(boosting_type='gbdt',num_leaves=15,max_depth=6,learning_rate=0.01,n_estimators=3500,subsample=0.8,tree_method = 'gpu_hist',
        #                   feature_fraction=0.7,random_state=seed,is_unbalance=True,eval_metric='auc')
        
        ctb_params = dict(iterations=3000,
                          learning_rate=0.01,
                          depth=8,
                          #cat_features = [],
                          l2_leaf_reg=30,
                          bootstrap_type='Bernoulli',
                          subsample=0.8,
                          loss_function='Logloss',
                          #eval_metric = 'AUC',
                          metric_period=100,
                          od_type='Iter',
                          od_wait=30,
                          task_type='GPU',
                          allow_writing_files=False,
                          )

        print("Feature Elimination Performing.")
        model2 = CatBoostClassifier(**ctb_params)
        
        print("================{}==============".format(seed))
        #kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        kfold = StratifiedGroupKFold(n_splits=folds)
        
        for fold_id, (trn_idx, val_idx) in enumerate(
                kfold.split(df_train[used_feat], df_train[ycol],groups=df_train['autherID'])):
        #for fold_id, (trn_idx, val_idx) in enumerate(
        #        kfold.split(df_train[used_feat], df_train[ycol],groups = df_train['autherID'])):
            X_train = df_train.iloc[trn_idx][used_feat]
            Y_train = df_train.iloc[trn_idx][ycol]

            X_val = df_train.iloc[val_idx][used_feat]
            Y_val = df_train.iloc[val_idx][ycol]
            #df_train.iloc[val_idx,'fold_id'] = fold_id

            print('\nFold_{} Training ================================\n'.format(fold_id + 1))


            lgb_model = model2.fit(X_train[used_feat],Y_train,
                              eval_set=[(X_val[used_feat], Y_val)],
                              #eval_metric = 'auc',
                              verbose=100,
                              #early_stopping_rounds=200
                                  )



            pred_val = lgb_model.predict_proba(X_val[used_feat])[:,1]


            df_oof.loc[val_idx, f"{ycol}_prob"  ] += pred_val


            pred_test = lgb_model.predict_proba(df_test[used_feat])[:,1]
            prediction[f"{ycol}_prob" ] += pred_test

            df_importance = pd.DataFrame({
                'column': used_feat,
                'importance': lgb_model.feature_importances_,
            })
            df_importance_list.append(df_importance)
            #break

            del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
            gc.collect()
            #break
            
       
        df_importance_dict[ycol] = df_importance_list.copy()
    #break   
LABEL_COLUME = ['label']  
prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = prediction[[f"{ycol}_prob" for ycol in LABEL_COLUME ]]/(folds*len(seeds))

df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] = df_oof[[f"{ycol}_prob" for ycol in LABEL_COLUME ]] / len(seeds)

In [ ]:
df_oof.to_feather('oof2/feat_select_cat_oof.feather')
prediction.to_feather('oof2/feat_select_cat_prediction.feather')